# Indexing

### Preface: chunking

we don't explicitly cover document chunking/splitting.

For an excelent review of document chunking, see the video from Greg Kamradt

https://www.youtube.com/watch?v=8OJC21T2SL4

Paper:
https://arxiv.org/abs/2312.06648

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = 'true'

os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

## Part 12: Multi-representation Indexing

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 5})

c:\Users\FIDELIS ALIU\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Failed to batch ingest runs: LangSmithConnectionError('Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000029009242300>, \'Connection to api.smith.langchain.com timed out. (connect timeout=10.0)\'))"))')
Failed to batch ingest runs: LangSmithConnectionError('Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000029009243650>, \'Connection to api.smith.langchain.com timed o

In [4]:
summaries

['This document explores the concept of LLM-powered autonomous agents, which are systems that use large language models (LLMs) as their core controllers. It delves into the key components of these agents, including planning, memory, and tool use.\n\n**Planning:**\n\n* **Task Decomposition:** LLMs can break down complex tasks into smaller, manageable subgoals using techniques like Chain of Thought (CoT) and Tree of Thoughts.\n* **Self-Reflection:** Agents can learn from past actions and refine their plans through self-criticism and self-reflection. Examples include ReAct, Reflexion, and Chain of Hindsight.\n\n**Memory:**\n\n* **Types of Memory:** The document draws parallels between human memory types (sensory, short-term, and long-term) and LLM capabilities.\n* **Maximum Inner Product Search (MIPS):** External vector stores are used for long-term memory, enabling fast retrieval of information using approximate nearest neighbors (ANN) algorithms like LSH, ANNOY, HNSW, FAISS, and ScaNN.\

In [10]:
from langchain.storage import InMemoryByteStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=os.getenv("GEMINI_API_KEY"))
)

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [11]:
query = "Memory in agents"

sub_docs = vectorstore.similarity_search(query, k=1)
sub_docs[0]

Document(metadata={'doc_id': 'ce8c89b7-b69f-451a-98e0-8dfd85935b21'}, page_content='This document explores the concept of LLM-powered autonomous agents, which are systems that use large language models (LLMs) as their core controllers. It delves into the key components of these agents, including planning, memory, and tool use.\n\n**Planning:**\n\n* **Task Decomposition:** LLMs can break down complex tasks into smaller, manageable subgoals using techniques like Chain of Thought (CoT) and Tree of Thoughts.\n* **Self-Reflection:** Agents can learn from past actions and refine their plans through self-criticism and self-reflection. Examples include ReAct, Reflexion, and Chain of Hindsight.\n\n**Memory:**\n\n* **Types of Memory:** The document draws parallels between human memory types (sensory, short-term, and long-term) and LLM capabilities.\n* **Maximum Inner Product Search (MIPS):** External vector stores are used for long-term memory, enabling fast retrieval of information using approx

In [12]:
retrieved_docs = retriever.get_relevant_documents(query, n_results=1)
retrieved_docs[0].page_content[0:500]

C:\Users\FIDELIS ALIU\AppData\Local\Temp\ipykernel_10868\494825069.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(query, n_results=1)


"\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n"

## Part 13: RAPTOR

https://www.youtube.com/watch?v=jbGchdTL7d0

Paper:
https://arxiv.org/abs/2401.18059

## Part 14: ColBERT

https://colbert.aiserv.cloud/
https://tim.simonwillison.net/llms/colbert-ragatouille
https://thenewstack.io/overcoming-the-limits-of-rag-with-colbert

In [ ]:
!pip install -U ragatouille

In [13]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

c:\Users\FIDELIS ALIU\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FIDELIS ALIU\.cache\huggingface\hub\models--colbert-ir--colbertv2.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[Sep 18, 05:26:52] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


c:\Users\FIDELIS ALIU\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\FIDELIS ALIU\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\cpp_extension.py:380: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')


CalledProcessError: Command '['where', 'cl']' returned non-zero exit status 1.

In [ ]:
import requests


def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.
    
    :param title: str - Title of the Wikipedia page.
    :return: str - full text content of the page as raw string
    """

    # Wikipeia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
        
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extrating page content
    page = next(iter(data["query"]["pages"].values()))

    return page["extract"] if "extract" in page else None

full_document = get_wikipedia_page("Hayao Miyazaki")

In [ ]:
RAG.index(
    collection=[full_document],
    index_name="Miyazaki-123",
    max_document_length=100,
    split_documents=True,
)

In [ ]:
results = RAG.search(query="What animation studio did Miyazaki found?", k=3)
results

In [ ]:
retriever = RAG.as_langchain_retriever(k=3)
retriever.invoke("What animation studio did Miyazaki found?")